In [1]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler
import random
import implicit
from sklearn import metrics
import matplotlib.pylab as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [76]:
pd.options.display.width = 100

## Key Summary

1. Based on Sijo's work, tried other algorithms of Implicit package.

2. Best auc_roc was 0.808 for LogisticMatrixFactorization (best number of factors is 35), but it's still lower than 0.836.

3. Test hypothesis: tried running recommendor on members who participate at least 3 groups. Because I though there was too few information in people who participate only 1 or 2 groups. The best result was 0.736 (with factor of 16), though it's lower than running on the whole data, but it's higher than 0.725 of recommending the most popular groups. 

4. Which means maybe we should recommend the most popular groups to those who only participate ne or two groups. And make a more personalized recommendor for more active members. 

## Next step:

try algorithms in this package: https://github.com/NicolasHug/Surprise

### Work flow
1. pre-processing data
    - merger member, event, rsvp
    - scale the number of events a member participated
    - get member_group matrix
2. split to train data set and test data set
    - my question: it looks like the author only take out the participated group
3. run the model
4. compare the result

[to do]
5. see if recommendation makes sense
    - extract one member 
    - see what group he/she participated
    - see what recommendation we made
    - try to tell a story about this
    - make few more examples

## Read-in data

In [2]:
%%time
#load the rsvp, member and events data 
rsvp = pd.read_csv('rsvps_all_new.csv')
member = pd.read_csv('members_new.csv',encoding='latin-1')
events = pd.read_csv('events_all_new.csv',encoding='latin-1')

<string>:2: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
<string>:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
<string>:2: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 54.3 s


In [3]:
groups = pd.read_csv('groups_austin.csv',encoding='latin-1')

## Summary of dfs

#### New

__rsvp__ we have 34592 event.id, 6740 member_id

__member__ we can get 299630 member_id, and 2273 group_id

__events__ we can get 35629 id, and 872 group_id, 872 group.name

#### Before

__rsvp__ we have 541 event_id, 2958 member_id

__member__ we can get 1087923 member_id, and 12546 group_id

__events__ we can get 5807 event_id, and 341 group_id, 341 group.name

## What's in our rsvp events

In [4]:
events.columns

Index(['created', 'description', 'duration', 'event_url', 'fee.accepts',
       'fee.amount', 'fee.currency', 'fee.description', 'fee.label',
       'fee.required', 'group.created', 'group.group_lat', 'group.group_lon',
       'group.id', 'group.join_mode', 'group.name', 'group.urlname',
       'group.who', 'headcount', 'how_to_find_us', 'id', 'maybe_rsvp_count',
       'name', 'photo_url', 'rsvp_limit', 'status', 'time', 'updated',
       'utc_offset', 'venue.address_1', 'venue.address_2', 'venue.city',
       'venue.country', 'venue.id', 'venue.lat',
       'venue.localized_country_name', 'venue.lon', 'venue.name',
       'venue.phone', 'venue.repinned', 'venue.state', 'venue.zip',
       'visibility', 'waitlist_count', 'why', 'yes_rsvp_count'],
      dtype='object')

In [5]:
mask = events['id'].map(lambda x: x in rsvp['event.id'].values)
events_masked = events[mask]

In [6]:
events_masked.shape

(34565, 46)

In [7]:
len(events_masked['name'].value_counts())

1980

In [8]:
# events_masked['venue.localized_country_name'].value_counts()
events_masked['venue.state'].value_counts()
# events_masked['created'].max() # date varies from 2017 to 2010

TX    25982
tx     2759
al      150
Te      149
CA       57
HI       13
FL        2
AR        2
WA        1
MN        1
VA        1
NM        1
Name: venue.state, dtype: int64

#### If the location vary that much, our reccomendation will not work. Because ppl in NY are not likely to participate events/ join groupd in CA. So they are basically two sets of data. We can improve our model by splitting them into different groups and train a model for each other to improve the performance.

## Preprocess data

In [59]:
%%time
#preprocesssing
main_data = pd.merge(rsvp, events, left_on= 'event.id', right_on='id', how='inner')
temp_data = main_data[['event.id', 'member.member_id', 'group.id_x', 'group.name']] 
main_data_2 = temp_data.groupby(['group.id_x','member.member_id']).size().reset_index()
main_data_2.columns = ['group_id', 'member_id','rsvp_sum']
main_data_2.head()

Wall time: 1.16 s


,group_id,member_id,rsvp_sum
0,10023,6682417,1
1,10023,13480894,1
2,10023,155423252,3
3,10023,161784042,1
4,10023,201386263,1


### Q: what is rsvp_sum?????
That means this member participated X events held by this group

#### In new data 17.7% of members joined more than 1 group

#### <font color='red'>In old data, 97% of members joined only one group...we need more data/ or data that focus on one city...</font>

In [10]:
main_data_2.shape

(8699, 3)

In [11]:
main_join_number = main_data_2['member_id'].value_counts()
len(main_join_number)

6740

In [12]:
np.sum(main_join_number > 1)/len(main_join_number)

0.17700296735905044

## Q: why scale?
The higher the scale, means the person participated more events held by the group

In [58]:
#standardizing to 1 - 10 scale 
main_data_2['rsvp_total'] = 1 + (main_data_2['rsvp_sum']-main_data_2['rsvp_sum'].min())*(10-1)/(main_data_2['rsvp_sum'].max()-main_data_2['rsvp_sum'].min())
main_data_2.head()

,group_id,member_id,rsvp_sum,rsvp_total


In [14]:
groups.shape

(2891, 37)

#### groups has category.name, but I can't match any variables in member
#### I also try to use topics to match, but didn't work out

In [15]:
groups['category.name'].value_counts().head()

tech                    566
career/business         465
health/wellbeing        218
socializing             168
new age/spirituality    143
Name: category.name, dtype: int64

In [16]:
a = groups[groups['id'] == 6671492]['topics'].values[0]
type(a)

str

## Chop off members who only participate one group or two groups

In [63]:
main_data = pd.merge(rsvp, events, left_on= 'event.id', right_on='id', how='inner')
temp_data = main_data[['event.id', 'member.member_id', 'group.id_x', 'group.name']] 
main_data_2 = temp_data.groupby(['group.id_x','member.member_id']).size().reset_index()
main_data_2.columns = ['group_id', 'member_id','rsvp_sum']
main_data_2['rsvp_total'] = 1 + (main_data_2['rsvp_sum']-main_data_2['rsvp_sum'].min())*(10-1)/(main_data_2['rsvp_sum'].max()-main_data_2['rsvp_sum'].min())
main_data_2 = main_data_2[main_data_2['rsvp_sum']<=2]

In [64]:
main_data_2.shape

(7230, 4)

## Produce sparse_item_user and sparse_user_item

In [65]:
data = main_data_2.dropna()
data = data.copy()

# Converting the numbers to categories to be used for creating the categorical codes to avoid using long hash keys 
data['member_id'] = data['member_id'].astype('category')
data['group_id'] = data['group_id'].astype('category')

#cat.codes creates a categorical id for the users and artists
data['user_id'] = data['member_id'].cat.codes
data['grp_id'] = data['group_id'].cat.codes

# The implicit library expects data as a item-user matrix so we
# create two matrices, one for fitting the model (item-user) 
# and one for recommendations (user-item)

# sparse_user: row is item
sparse_item_user = sparse.csr_matrix((data['rsvp_total'].astype(float), (data['grp_id'], data['user_id'])))

# sparse_user: row is user
sparse_user_item = sparse.csr_matrix((data['rsvp_total'].astype(float), (data['user_id'], data['grp_id'])))

In [ ]:
### codes from reference...helps me to understand what we're trying to do here

# customers = list(np.sort(grouped_purchased.CustomerID.unique())) # Get our unique customers
# products = list(grouped_purchased.StockCode.unique()) # Get our unique products that were purchased
# quantity = list(grouped_purchased.Quantity) # All of our purchases

# rows = grouped_purchased.CustomerID.astype('category', categories = customers).cat.codes 
# # Get the associated row indices
# cols = grouped_purchased.StockCode.astype('category', categories = products).cat.codes 
# # Get the associated column indices
# purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

### To check sparsity
According to https://jessesw.com/Rec-System/ _"For collaborative filtering to work, the maximum sparsity you could get away with would probably be about 99.5% or so. We are well below this, so we should be able to get decent results."_

In [66]:
matrix_size = sparse_user_item.shape[0]*sparse_user_item.shape[1] # Number of possible interactions in the matrix
num_purchases = len(sparse_user_item.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

99.81142852671691

## Produce train data

In [21]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of item,user index into list
    
    random.seed(0) # Set the random seed to zero for reproducibility
    
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of item-user pairs without replacement

    item_inds = [index[0] for index in samples] # Get the item row indices

    user_inds = [index[1] for index in samples] # Get the user column indices
    
    training_set[item_inds, user_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    mask = {}
    #reference dictionary
    
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user columns that were altered

## Q: Why do we use item_user here?
Guess it's because followed http://www.benfrederickson.com/matrix-factorization/

Note in https://jessesw.com/Rec-System/, the author actually used user as row. Which I think makes more sense to our case, since we are making reccomendation to users. 

But here says the model takes item_user matrix https://implicit.readthedocs.io/en/latest/als.html, therefore I believe Sijo did it right.

In [67]:
# 20% of the data has been masked for this exercise
product_train, product_test, product_users_altered = make_train(sparse_item_user, pct_test = 0.2)

In [49]:
len(product_users_altered) ## we randomly chose 20% of ppl and take out the members' records

280

# Train a model

### Q: what is 'alpha' and 'factors'
can find answer here: https://jessesw.com/Rec-System/

In [68]:
# Initialize the als model and fit it using the sparse item-user matrix
# Parameters that we have chosen
# 1. factors = 20 -- Latent factors for user and item vectors
# 2. iterations = 20 -- Number of iterations to use while fitting the data
# 3. regularization = 0.1 -- regularization constant to be used in the cost function

# model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=40)

model = implicit.lmf.LogisticMatrixFactorization(factors=16, regularization=0.1, iterations=40)

# Calculate the confidence by multiplying it by our alpha value.(alpha value corresponds to the confidence metric 
# that we discussed earlier)

alpha_val = 15
data_conf = (product_train * alpha_val).astype('double')

# We have used an alpha_val of 15 after performing some iterations with different alpha values
#Fit the model
model.fit(product_train)

In [69]:
item_vecs = model.item_factors
user_vecs = model.user_factors

print('Shape of group vector matrix : ', item_vecs.shape)
print('Shape of member vector matrix : ', user_vecs.shape)

Shape of group vector matrix :  (678, 18)
Shape of member vector matrix :  (5655, 18)


## Evaluate the model

In [70]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 1)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_column = training_set[:,user].toarray().reshape(-1) # Get the training set column
        zero_inds = np.where(training_column == 0) # Find where the interaction had not yet occurred
        
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[:,user].toarray()[zero_inds,0].reshape(-1)
        
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
#         pop = pop/np.max(pop)
        
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score

#         print('user', user)
#         print('zero_inds', zero_inds)
#         print('pred', pred)
#         print('actual', actual)
#         print('pop', pop)
#         print('pred auc', auc_score(pred, actual))       
#         print('pop auc', auc_score(pop, actual))
    # End users iteration
#         print('======================')
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

Just chek pop_items

In [97]:
# pop_items = np.array(product_test.sum(axis = 1)).reshape(-1)

# user = 2052
# training_column = product_train[:,user].toarray().reshape(-1)
# zero_inds = np.where(training_column == 0)
# pop = pop_items[zero_inds]
# # pop = pop[ pop == np.max(pop)]

# pop_items

## Our recommendation is not good. 
The first number is the AUC of our model, the second is AUC if we recommend the most popular event.

#### below is ALS

In [103]:
# calc_mean_auc(product_train, product_users_altered,
#               [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)

(0.54, 0.836)

#### below is BayesianPersonalizedRanking

In [110]:
# calc_mean_auc(product_train, product_users_altered,
#               [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)

(0.509, 0.836)

#### LogisticMatrixFactorization

In [71]:
calc_mean_auc(product_train, product_users_altered,
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)

(0.793, 0.829)

In [53]:
%%time
auc = dict()
for i in range(2,51):
    model = implicit.lmf.LogisticMatrixFactorization(factors=i, regularization=0.1, iterations=40)
    alpha_val = 15
    data_conf = (product_train * alpha_val).astype('double')
    model.fit(product_train, show_progress=False)
    item_vecs = model.item_factors
    user_vecs = model.user_factors
    
    auc[i] = calc_mean_auc(product_train, product_users_altered,
                  [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)[0]

Wall time: 46.4 s


In [54]:
check = pd.Series(auc)
check[check == check.max()]

16    0.739
dtype: float64

#### LogisticMatrixFactorization for rsvp_sum >= 3

In [153]:
calc_mean_auc(product_train, product_users_altered,
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)

(0.669, 0.725)

In [39]:
%%time
auc = dict()
for i in range(2,51):
    model = implicit.lmf.LogisticMatrixFactorization(factors=i, regularization=0.1, iterations=40)
    alpha_val = 15
    data_conf = (product_train * alpha_val).astype('double')
    model.fit(product_train, show_progress=False)
    item_vecs = model.item_factors
    user_vecs = model.user_factors
    
    auc[i] = calc_mean_auc(product_train, product_users_altered,
                  [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)[0]

Wall time: 44.2 s


In [40]:
check = pd.Series(auc)
check[check == check.max()]

14    0.727
dtype: float64

## A show case of our recommender

In [73]:
data.columns

Index(['group_id', 'member_id', 'rsvp_sum', 'rsvp_total', 'user_id', 'grp_id'], dtype='object')

In [74]:
events.columns

Index(['created', 'description', 'duration', 'event_url', 'fee.accepts',
       'fee.amount', 'fee.currency', 'fee.description', 'fee.label',
       'fee.required', 'group.created', 'group.group_lat', 'group.group_lon',
       'group.id', 'group.join_mode', 'group.name', 'group.urlname',
       'group.who', 'headcount', 'how_to_find_us', 'id', 'maybe_rsvp_count',
       'name', 'photo_url', 'rsvp_limit', 'status', 'time', 'updated',
       'utc_offset', 'venue.address_1', 'venue.address_2', 'venue.city',
       'venue.country', 'venue.id', 'venue.lat',
       'venue.localized_country_name', 'venue.lon', 'venue.name',
       'venue.phone', 'venue.repinned', 'venue.state', 'venue.zip',
       'visibility', 'waitlist_count', 'why', 'yes_rsvp_count'],
      dtype='object')

In [79]:
test=pd.merge(data, events[['group.name','group.id']], left_on = 'group_id', right_on = 'group.id', how = 'inner')
test=test.drop_duplicates()

# raeding chinese books group - 26243452

group_id = 300
n_similar = 10 # getting the top ten similar items
# print ("------------Actual group-------------------")
# print (test[test['grp_id'] == group_id]['group.name'])
# print ("------------Actual group-------------------")

# Use implicit to get similar items.
similar = model.similar_items(group_id, 10)
# Print the names of our most similar artists
for group in similar:
    idx, score = group
    print (test['group.name'].loc[test.grp_id == idx].iloc[0])

Austin Office 365 and SharePoint User Group
Austin Virtual Bible Study Meetup
NBX - Austin Professional Networking Group
Club Turnttt
Beat Burnout With Emotional Energetic Release
Austin Sailing Adventures
Austin Reiki Classes/Attunement Meetup
Weeknight Wordsmiths
The Austin Physician Network
Mindfulness in Nature


In [27]:
#top member
test['rank'] = test.rsvp_total.rank(ascending = False, method='max') 
test_1 = test[test['rank'] <10]
test_1.head(25)

,group_id,member_id,rsvp_sum,rsvp_total,user_id,grp_id,group.name,rank
32159,2148441,5189189,42,8.530612,86,17,SF Free School,3.0
43415,2148441,88770122,39,7.979592,719,17,SF Free School,5.0
108186,18177413,192517329,33,6.877551,1366,35,Bryant Park Toastmasters,7.0
108286,18177413,195703963,50,10.000000,1429,35,Bryant Park Toastmasters,1.0
108786,18177413,203392611,31,6.510204,1600,35,Bryant Park Toastmasters,9.0
109386,18177413,224325846,45,9.081633,1940,35,Bryant Park Toastmasters,2.0
114605,18468560,184845608,33,6.877551,1187,40,Lexington/Vanderbilt toastmasters public speak...,7.0
120420,26243452,78357452,31,6.510204,675,46,Reading Chinese Books,9.0
121470,26243452,234733619,39,7.979592,2205,46,Reading Chinese Books,5.0


In [103]:
member_id = 6192675
user_id = test[test['member_id'] == member_id]['user_id'][0]

# Use the implicit recommender.
recommended = model.recommend(user_id, sparse_user_item,N = 10,filter_already_liked_items = False)

artists = []
scores = []

# Get artist names from ids
for item in recommended:
    idx, score = item
    artists.append(test['group.name'].loc[test.grp_id == idx].iloc[0])
    scores.append(score)

# Create a dataframe of artist names and scores
recommendations = pd.DataFrame({'artist': artists, 'score': scores})

recommendations

IndexError: index out of bounds

In [94]:
# product_train, product_test, product_users_altered

In [96]:
product_train.shape

(678, 5655)

In [100]:
product_test.shape

(678, 5655)

In [99]:
len(product_users_altered)

1353